In [1]:
from sympy import *
from IPython.display import display
init_printing(use_latex='mathjax')

# Series Solutions to ODEs

Before we bein asymptotics in earnest, we explain how Sympy can be used to compute series solutions. Most of the techniques used in later codes make themselves present here as well. The idea of this first code is to generate a series solution to the equation $$\ddot{y} - y = 0.$$ First we must define our variables $x$ which is a *symbol* a fundamental building block of Sympy's computer algebra system. Then we define $N$ to indicate the number of terms in our series, followed by the symbol array $\{a_i\}_{i=0}^{N-1}$, built out of an f-string on top of slice notation. Finally, we define $y=\sum_{i=0}^{N-1}a_ix^i.$ Note, the variable array `a_` is simply a personal convention to match $\LaTeX$/Markdown syntax. 

The function call `diff(x,y,z)` indicates derivatives with respect to variables x, y, z. There are other ways to set up derivatives in Sympy, but this is the most common, and the one used the most.

In summary what this code does is that it takes the series solution for $y$, defines the equation with that solution embedded into it. Then the equation is internally transformed before we call `collect(x)` which gathers terms by their powers of $x$, `evaluate=False` tells us that we want each of these terms separated into a dictionary whose keys are the powers of $x$ and the values are those coefficients, which is then printed.

The idea behind a series solution is that each of these coefficients should become 0, so that the equation as a whole would be uniformly 0. As such, that is precisely what we do in the following for loop. We also define a dictionary `replacements` where we can store previously solved $a_i$ values for use in later equations. It is important to note that we solve all $a_i$ in terms of $a_0,a_1$ since these will are the leading coefficients of the two linearly independent solutions.

Finally, we substitute all the variables we solved for back into our definition of $y$ and print.

In [2]:
x = symbols('x')
N = 10
a_ = symbols(f'a0:{N}')
y = sum([a_[i]*x**i for i in range(N)])

eqn = y.diff(x,x) - y
eqn = eqn.expand().powsimp().collect(x,evaluate=False)
display(eqn)
replacements = {}
for i in range(N-2):
    replacements[a_[i+2]] = solve(eqn[x**i].subs(replacements),a_[i+2])[0]
    
display(y.subs(replacements))


⎧                               2                3                4            ↪
⎨1: -a₀ + 2⋅a₂, x: -a₁ + 6⋅a₃, x : -a₂ + 12⋅a₄, x : -a₃ + 20⋅a₅, x : -a₄ + 30⋅ ↪
⎩                                                                              ↪

↪      5                6                7                8        9     ⎫
↪ a₆, x : -a₅ + 42⋅a₇, x : -a₆ + 56⋅a₈, x : -a₇ + 72⋅a₉, x : -a₈, x : -a₉⎬
↪                                                                        ⎭

    8       6       4       2            9        7       5       3       
a₀⋅x    a₀⋅x    a₀⋅x    a₀⋅x         a₁⋅x     a₁⋅x    a₁⋅x    a₁⋅x        
───── + ───── + ───── + ───── + a₀ + ────── + ───── + ───── + ───── + a₁⋅x
40320    720     24       2          362880   5040     120      6         

## Frobenius Method

The Frobenius method operates in a similar method as the above power series solution, however we instead use the more generic power series $y=x^\alpha\sum_{i=0}^{N-1}a_ix^i,$ which we use to solve the differential equation $$x\ddot{y} + \dot{y} + xy = 0.$$ Since `collect` only works for integer powers of $x$, we factor out the generic $x^\alpha$ that appears in all terms, imitatin a division by $x^\alpha$ once all the derivatives have done their part. 

We then start by solving the *indicial polynomial* which occupies the most negative order of $x,$ here $x^{-1}.$ This gives the valid values of $\alpha.$ We thn solve the coefficients like before, then printing the full $y$ in the end.

In [17]:
x, alpha = symbols('x alpha')
N = 10
a_ = symbols(f'a0:{N}')
y = sum([a_[i]*x**(i+alpha) for i in range(N)])

eqn = x*y.diff(x,x) + y.diff(x) + x*y
eqn = eqn.expand().subs(x**alpha,1).collect(x,evaluate=False)
display(eqn)

replacements = [{alpha:a} for a in solve(eqn[x**-1],alpha)]

for r in replacements:
    for i in range(N-1):
        r[a_[i+1]] = solve(eqn[x**i].subs(r),a_[i+1])[0]
    display(y.subs(r))


⎧       2                1      2              2                   2           ↪
⎨1: a₁⋅α  + 2⋅a₁⋅α + a₁, ─: a₀⋅α , x: a₀ + a₂⋅α  + 4⋅a₂⋅α + 4⋅a₂, x : a₁ + a₃⋅ ↪
⎩                        x                                                     ↪

↪  2                   3           2                    4           2          ↪
↪ α  + 6⋅a₃⋅α + 9⋅a₃, x : a₂ + a₄⋅α  + 8⋅a₄⋅α + 16⋅a₄, x : a₃ + a₅⋅α  + 10⋅a₅⋅ ↪
↪                                                                              ↪

↪             5           2                     6           2                  ↪
↪ α + 25⋅a₅, x : a₄ + a₆⋅α  + 12⋅a₆⋅α + 36⋅a₆, x : a₅ + a₇⋅α  + 14⋅a₇⋅α + 49⋅a ↪
↪                                                                              ↪

↪     7           2                     8           2                     9    ↪
↪ ₇, x : a₆ + a₈⋅α  + 16⋅a₈⋅α + 64⋅a₈, x : a₇ + a₉⋅α  + 18⋅a₉⋅α + 81⋅a₉, x : a ↪
↪                                                                              ↪

↪     10    ⎫
↪ ₈, x  : 

    8        6       4       2     
a₀⋅x     a₀⋅x    a₀⋅x    a₀⋅x      
────── - ───── + ───── - ───── + a₀
147456   2304     64       4       